In [46]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film']) # Karena kita hanya akan menggunakan sinopsis

In [47]:
df.head()

,judul_film,genre
0,Sunan Kalijaga,Drama
1,Gie,Drama
2,Guru Bangsa Tjokroaminoto,Drama
3,POL Movie,Drama
4,Sang pencerah,Drama


In [48]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,judul_film,Drama,Horor,Komedi,Laga,Romantis
0,Sunan Kalijaga,1.0,0.0,0.0,0.0,0.0
1,Gie,1.0,0.0,0.0,0.0,0.0
2,Guru Bangsa Tjokroaminoto,1.0,0.0,0.0,0.0,0.0
3,POL Movie,1.0,0.0,0.0,0.0,0.0
4,Sang pencerah,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1000,Winter in Tokyo,0.0,0.0,0.0,0.0,1.0
1001,Petualang Cinta,0.0,0.0,0.0,0.0,1.0
1002,Last Night (III),0.0,0.0,0.0,0.0,1.0
1003,Path of Light,0.0,0.0,0.0,0.0,1.0


In [49]:
# Agar dapat diproses model, ubah nilai-nilai tersebut ke numpy array
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [50]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [51]:
# Tokenisasi
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih)
tokenizer.fit_on_texts(sinopsis_test)

sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)

padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)

In [52]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax'),
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [54]:
num_epochs = 30

history = model.fit(
    padded_latih,
    label_latih,
    epochs=num_epochs,
    validation_data=(padded_test, label_test),
    verbose=2
)

Epoch 1/30
26/26 - 4s - loss: 1.6100 - accuracy: 0.1978 - val_loss: 1.6102 - val_accuracy: 0.2090 - 4s/epoch - 152ms/step
Epoch 2/30
26/26 - 1s - loss: 1.6086 - accuracy: 0.2090 - val_loss: 1.6081 - val_accuracy: 0.3184 - 779ms/epoch - 30ms/step
Epoch 3/30
26/26 - 1s - loss: 1.5991 - accuracy: 0.2687 - val_loss: 1.5936 - val_accuracy: 0.3134 - 793ms/epoch - 31ms/step
Epoch 4/30
26/26 - 1s - loss: 1.5068 - accuracy: 0.3719 - val_loss: 1.5833 - val_accuracy: 0.2886 - 863ms/epoch - 33ms/step
Epoch 5/30
26/26 - 0s - loss: 1.2381 - accuracy: 0.4490 - val_loss: 1.6348 - val_accuracy: 0.2985 - 445ms/epoch - 17ms/step
Epoch 6/30
26/26 - 0s - loss: 1.0072 - accuracy: 0.5709 - val_loss: 1.6832 - val_accuracy: 0.3234 - 431ms/epoch - 17ms/step
Epoch 7/30
26/26 - 0s - loss: 0.8350 - accuracy: 0.6604 - val_loss: 1.9630 - val_accuracy: 0.3284 - 435ms/epoch - 17ms/step
Epoch 8/30
26/26 - 0s - loss: 0.7460 - accuracy: 0.6915 - val_loss: 2.1669 - val_accuracy: 0.3781 - 455ms/epoch - 18ms/step
Epoch 9/30